In [38]:
# Load packages

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from covid19model.models.models import COVID19_SEIQRD_spatial_stratified_rescaling
from covid19model.models.utils import initialize_COVID19_SEIQRD_stratified_vacc

# Import time-dependent parameter functions for resp. P, Nc, alpha, N_vacc, season_factor
from covid19model.models.time_dependant_parameter_fncs import make_mobility_update_function, \
                                                          make_contact_matrix_function, \
                                                          make_VOC_function, \
                                                          make_vaccination_function, \
                                                          make_seasonality_function
# Import packages containing functions to load in data used in the model and the time-dependent parameter functions
from covid19model.data import mobility, sciensano, model_parameters, VOC

In [30]:
# load data

# Population size, interaction matrices and the model parameters
initN, Nc_dict, params, CORE_samples_dict = model_parameters.get_COVID19_SEIQRD_parameters(spatial='prov')

# Google Mobility data (for social contact Nc)
df_google = mobility.get_google_mobility_data(update=False, provincial=True)

# Load and format mobility dataframe (for mobility place)
proximus_mobility_data = mobility.get_proximus_mobility_data('prov')

# Variants of concern
VOCs = ['WT', 'abc', 'delta']
VOC_logistic_growth_parameters, VOC_params = model_parameters.get_COVID19_SEIQRD_VOC_parameters(initN, params['h'], VOCs=VOCs)
params.update(VOC_params)

# Load and format local vaccination data, which is also under the sciensano object
public_spatial_vaccination_data = sciensano.get_public_spatial_vaccination_data(update=False,agg='prov')

In [32]:
# Delete parameters that are superfluous for the new approach
superfluous_params = ['f_immune_escape', 'e_s', 'e_h', 'e_i', 'doses', 'd_vacc', 'N_vacc', 'daily_doses', 'delay_immunity', 'vacc_order', 'stop_idx', 'refusal']
for key in superfluous_params:
    if key in params:
        params.pop(key)

In [41]:
# Time-dependent parameter functions

# Time-dependent social contact matrix over all policies, updating Nc
policy_function = make_contact_matrix_function(df_google, Nc_dict).policies_all_spatial
policy_function_work = make_contact_matrix_function(df_google, Nc_dict).policies_all_work_only

# Time-dependent mobility function, updating P (place)
mobility_function = make_mobility_update_function(proximus_mobility_data).mobility_wrapper_func

# Time-dependent VOC function, updating alpha
VOC_function = make_VOC_function(VOC_logistic_growth_parameters)

# Time-dependent vaccination-induced rescaling parameter functions, updating E_susc^g, E_inf^g, E_hosp^g
E_susc_function, E_inf_function, E_hosp_function = make_vaccination_rescaling_functions(public_spatial_vaccination_data['INCIDENCE'],
                                                                                       VOC_function)

# Time-dependent seasonality function, updating season_factor
seasonality_function = make_seasonality_function()

In [46]:
VOC_function(pd.Timestamp(2021, 6, 1), 0, 0)[0]

array([0.        , 0.93339196, 0.06660804])